In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os
from sklearn.metrics import mean_squared_error
import datetime
import tensorflow as tf
from tensorflow.contrib import learn
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
import cPickle
import gzip
# import seaborn as sns
from sklearn import preprocessing
import random
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# 制作数据集
path = '/home/chocolate/LSTM-source/Main_product_added_features/'
seq_new_15 = []

data = pd.read_csv(path + 'IF_main_cleaned_addfeature.csv')
data_new = data.drop(data.columns[11:],axis = 1)

# 前25分钟和最后5分钟的某些特征为0值，会影响结果，予以删除
data_new = data_new.ix[25:, 1:]
data_new = data_new[:-5]
data_new.index = range(len(data_new))

In [3]:
#   数据归一化
data_array_per = np.array(data_new)
min_max_scaler = preprocessing.MinMaxScaler()
data_array_per = min_max_scaler.fit_transform(data_array_per)
#  组时间序列
for k in range(len(data_array_per)):
    if k < len(data_array_per) and k >= 15:
        seq_new_15.append(data_array_per[k - 15:k])
seq_new_15 = np.array(seq_new_15)
print seq_new_15.shape

(146384, 15, 10)


In [117]:
# 打数据集标签,根据归一化后的涨跌情况打标签。
seq_label = []

# print u'IF合约:'
data_IF = pd.read_csv(path + 'IF_main_cleaned_addfeature.csv')
data_per_new = data_IF[40:]
data_per_new = data_per_new[:-5]
 #   数据归一化
data_array_per = np.array(data_per_new)
min_max_scaler = preprocessing.MinMaxScaler()
data_array_per = min_max_scaler.fit_transform(data_array_per)
#     print data_array_per
data_per = pd.DataFrame(data_array_per,columns = data_per_new.columns)
# print data_per['RaiseDown_1'].describe()

In [5]:
data_per

,Unnamed: 0,Latestprice,MaxPrice,MinPrice,Stockup,Volume,First.Latestprice,Last.Buy1price,Last.Buy1quantity,Last.Sell1price,...,RaiseDown_1,RaiseDown_2,RaiseDown_3,RaiseDown_4,RaiseDown_5,MA_5_PRED,MA_12_PRED,MA_26_PRED,boll_up_PRED,boll_down_PRED
0,0.000000,0.012069,0.012305,0.012521,0.670048,0.073440,0.012752,0.397112,0.003779,0.396738,...,0.498307,0.436031,0.461457,0.462264,0.447239,0.013132,0.013043,0.012271,0.015503,0.013822
1,0.000007,0.019528,0.019579,0.019998,0.656203,0.050382,0.019978,0.401703,0.000472,0.401324,...,0.436031,0.461457,0.462264,0.447239,0.460851,0.012887,0.012967,0.012192,0.015502,0.013670
2,0.000014,0.014313,0.014730,0.014770,0.641672,0.097200,0.015120,0.398519,0.008385,0.398143,...,0.461457,0.462264,0.447239,0.460851,0.475942,0.012765,0.012522,0.011939,0.014438,0.013860
3,0.000020,0.014555,0.014609,0.014770,0.665936,0.046146,0.014817,0.398667,0.000472,0.398291,...,0.462264,0.447239,0.460851,0.475942,0.483742,0.013266,0.012103,0.011986,0.013400,0.014075
4,0.000027,0.014798,0.014851,0.014892,0.648389,0.049849,0.014938,0.398778,0.014289,0.398402,...,0.447239,0.460851,0.475942,0.483742,0.435519,0.013217,0.011822,0.011790,0.013432,0.013479
5,0.000034,0.011826,0.012002,0.012278,0.657848,0.038656,0.012448,0.396964,0.000118,0.396627,...,0.460851,0.475942,0.483742,0.435519,0.461256,0.011654,0.011761,0.011614,0.013451,0.013337
6,0.000041,0.011765,0.011942,0.012157,0.656477,0.036496,0.012205,0.396964,0.000472,0.396627,...,0.475942,0.483742,0.435519,0.461256,0.448739,0.011739,0.011694,0.011574,0.013380,0.013276
7,0.000048,0.014737,0.014851,0.015074,0.658807,0.055150,0.015302,0.398704,0.000236,0.398402,...,0.483742,0.435519,0.461256,0.448739,0.477570,0.012680,0.011628,0.011704,0.013331,0.013193
8,0.000055,0.019225,0.019458,0.019511,0.660727,0.127777,0.019978,0.401518,0.000827,0.401139,...,0.435519,0.461256,0.448739,0.477570,0.484647,0.012496,0.011490,0.011657,0.013208,0.013042
9,0.000061,0.013888,0.014366,0.014345,0.633996,0.081014,0.014452,0.398260,0.001299,0.397884,...,0.461256,0.448739,0.477570,0.484647,0.460755,0.012961,0.011403,0.011605,0.013115,0.012961


### 分类区间依据，来源于LSTM_V5.0

In [ ]:
 for i in data_per['RaiseDown_1']:
    if i > 0.68:
        seq_label.append([1,0,0,0,0,0,0])   #大涨
    elif i > 0.58:
        seq_label.append([0,1,0,0,0,0,0])   #小涨
    elif i> 0.48: 
        seq_label.append([0,0,1,0,0,0,0])   #平稳
    elif i > 0.44:
        seq_label.append([0,0,0,1,0,0,0])   #小跌
    elif i > 0.34:
        seq_label.append([0,0,0,0,1,0,0])   #大跌
    elif i > 0.24:
        seq_label.append([0,0,0,0,0,1,0])   #大跌
    else:
        seq_label.append([0,0,0,0,0,0,1])   #大跌

print u'涨幅3：',seq_label.count([1,0,0,0,0,0,0]),seq_label.count([1,0,0,0,0,0,0]) /146419.0
print u'涨幅2：',seq_label.count([0,1,0,0,0,0,0]),seq_label.count([0,1,0,0,0,0,0]) /146419.0
print u'涨幅1：',seq_label.count([0,0,1,0,0,0,0]),seq_label.count([0,0,1,0,0,0,0]) /146419.0
print u'平 稳：',seq_label.count([0,0,0,1,0,0,0]),seq_label.count([0,0,0,1,0,0,0]) /146419.0
print u'跌幅1：',seq_label.count([0,0,0,0,1,0,0]),seq_label.count([0,0,0,0,1,0,0]) /146419.0
print u'跌幅2：',seq_label.count([0,0,0,0,0,1,0]),seq_label.count([0,0,0,0,0,1,0]) /146419.0
print u'跌幅3：',seq_label.count([0,0,0,0,0,0,1]),seq_label.count([0,0,0,0,0,0,1]) /146419.0

In [110]:
# 回归5类，按照分类区间分割，分为大涨、小涨、平稳、小跌、大跌
# 表示出涨跌幅位于哪个区间
minute1 = pd.cut(data_per['RaiseDown_1'],[-100,.34,.44,.48,.58,1])
minute2 = pd.cut(data_per['RaiseDown_2'],[-100,.34,.44,.48,.58,1])
minute3 = pd.cut(data_per['RaiseDown_3'],[-100,.34,.44,.48,.58,1])
minute4 = pd.cut(data_per['RaiseDown_4'],[-100,.34,.44,.48,.58,1])
minute5 = pd.cut(data_per['RaiseDown_5'],[-100,.34,.44,.48,.58,1])
pd.concat([minute1,minute2,minute3,minute4,minute5],axis=1)


,RaiseDown_1,RaiseDown_2,RaiseDown_3,RaiseDown_4,RaiseDown_5
0,"(0.48, 0.58]","(0.34, 0.44]","(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]"
1,"(0.34, 0.44]","(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]"
2,"(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]"
3,"(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]","(0.48, 0.58]"
4,"(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]","(0.48, 0.58]","(0.34, 0.44]"
5,"(0.44, 0.48]","(0.44, 0.48]","(0.48, 0.58]","(0.34, 0.44]","(0.44, 0.48]"
6,"(0.44, 0.48]","(0.48, 0.58]","(0.34, 0.44]","(0.44, 0.48]","(0.44, 0.48]"
7,"(0.48, 0.58]","(0.34, 0.44]","(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]"
8,"(0.34, 0.44]","(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]","(0.48, 0.58]"
9,"(0.44, 0.48]","(0.44, 0.48]","(0.44, 0.48]","(0.48, 0.58]","(0.44, 0.48]"


In [123]:
# 计算出后5分钟内，涨跌情况分别属于每个区间的数量
data_temp = pd.concat([minute1,minute2,minute3,minute4,minute5],axis=1)
df = DataFrame()
from collections import Counter
for data_temp1 in data_temp.values:
    #print Counter(data333)
    df = pd.concat([df,DataFrame(dict(Counter(data_temp1)),index=[0])],ignore_index=True)
df.head()
# data_per.drop('raise2_count',axis=1,inplace=True)
df.columns = ['down2_count','down1_count','flat_count','raise1_count','raise2_count']
data = data_per.join(df)
data = data.replace(np.nan,0)

In [124]:
path = '/home/chocolate/LSTM-source/Main_product_added_features/'
data.to_csv(path + 'IF_main_cleaned_addcount.csv')

In [125]:
data = pd.read_csv(path + 'IF_main_cleaned_addcount.csv')
data.ix[:,1:]

,Unnamed: 0.1,Latestprice,MaxPrice,MinPrice,Stockup,Volume,First.Latestprice,Last.Buy1price,Last.Buy1quantity,Last.Sell1price,...,MA_5_PRED,MA_12_PRED,MA_26_PRED,boll_up_PRED,boll_down_PRED,down2_count,down1_count,flat_count,raise1_count,raise2_count
0,0.000000,0.012069,0.012305,0.012521,0.670048,0.073440,0.012752,0.397112,0.003779,0.396738,...,0.013132,0.013043,0.012271,0.015503,0.013822,0,1,3,1,0
1,0.000007,0.019528,0.019579,0.019998,0.656203,0.050382,0.019978,0.401703,0.000472,0.401324,...,0.012887,0.012967,0.012192,0.015502,0.013670,0,1,4,0,0
2,0.000014,0.014313,0.014730,0.014770,0.641672,0.097200,0.015120,0.398519,0.008385,0.398143,...,0.012765,0.012522,0.011939,0.014438,0.013860,0,0,5,0,0
3,0.000020,0.014555,0.014609,0.014770,0.665936,0.046146,0.014817,0.398667,0.000472,0.398291,...,0.013266,0.012103,0.011986,0.013400,0.014075,0,0,4,1,0
4,0.000027,0.014798,0.014851,0.014892,0.648389,0.049849,0.014938,0.398778,0.014289,0.398402,...,0.013217,0.011822,0.011790,0.013432,0.013479,0,1,3,1,0
5,0.000034,0.011826,0.012002,0.012278,0.657848,0.038656,0.012448,0.396964,0.000118,0.396627,...,0.011654,0.011761,0.011614,0.013451,0.013337,0,1,3,1,0
6,0.000041,0.011765,0.011942,0.012157,0.656477,0.036496,0.012205,0.396964,0.000472,0.396627,...,0.011739,0.011694,0.011574,0.013380,0.013276,0,1,3,1,0
7,0.000048,0.014737,0.014851,0.015074,0.658807,0.055150,0.015302,0.398704,0.000236,0.398402,...,0.012680,0.011628,0.011704,0.013331,0.013193,0,1,3,1,0
8,0.000055,0.019225,0.019458,0.019511,0.660727,0.127777,0.019978,0.401518,0.000827,0.401139,...,0.012496,0.011490,0.011657,0.013208,0.013042,0,1,3,1,0
9,0.000061,0.013888,0.014366,0.014345,0.633996,0.081014,0.014452,0.398260,0.001299,0.397884,...,0.012961,0.011403,0.011605,0.013115,0.012961,0,0,4,1,0


In [133]:
seq_label = []
data_temp = data
for k in range(len(data_temp)):
    # 开多仓条件
    if data_temp['raise2_count'].iloc[k] >= 2 and data_temp['down1_count'].iloc[k] <= 2 and data_temp['down2_count'].iloc[k] == 0:
        seq_label.append([1,0,0])
    elif data_temp['raise2_count'].iloc[k] >= 1 and data_temp['down1_count'].iloc[k] <= 1 and data_temp['down2_count'].iloc[k] == 0:
        seq_label.append([1,0,0])
    elif data_temp['raise1_count'].iloc[k] >= 3 and data_temp['down1_count'].iloc[k] <= 1 and data_temp['down2_count'].iloc[k] == 0:
        seq_label.append([1,0,0])
    # 开空仓条件
    elif data_temp['down2_count'].iloc[k] >= 2 and data_temp['raise1_count'].iloc[k] <= 2 and data_temp['raise2_count'].iloc[k] == 0:
        seq_label.append([0,0,1])
    elif data_temp['down2_count'].iloc[k] >= 1 and data_temp['raise1_count'].iloc[k] <= 1 and data_temp['raise2_count'].iloc[k] == 0:
        seq_label.append([0,0,1])
    elif data_temp['down1_count'].iloc[k] >= 3 and data_temp['raise1_count'].iloc[k] <= 1 and data_temp['raise2_count'].iloc[k] == 0:
        seq_label.append([0,0,1])
    else:
        seq_label.append([0,1,0])
        
print u'开多仓：',seq_label.count([1,0,0]),seq_label.count([1,0,0]) /146384.0
print u'不开仓：',seq_label.count([0,1,0]),seq_label.count([0,1,0]) /146384.0
print u'开空仓：',seq_label.count([0,0,1]),seq_label.count([0,0,1]) /146384.0

    

开多仓： 8229 0.0562151601268
不开仓： 132069 0.902209257842
开空仓： 6086 0.0415755820308


In [132]:
arr_label=np.array(seq_label)
arr_label.shape

(146384, 3)

In [134]:
from random import shuffle
zip_seq = zip(seq_new_15,seq_label)
shuffle(zip_seq)
seq_new_15,seq_label = zip(*zip_seq)
seq_new_15 = np.array(seq_new_15) 
seq_label = np.array(seq_label)
print seq_new_15.shape, seq_label.shape

(146384, 15, 10) (146384, 3)


In [135]:
train_data = seq_new_15[:102500]
train_label = seq_label[:102500]

test_data = seq_new_15[102500:]
test_label = seq_label[102500:]
train_data.shape, test_label.shape

((102500, 15, 10), (43884, 3))

In [136]:
# 保存训练集和测试集
path_new = '/home/chocolate/LSTM-source/train_test_data/'
f1 = file(path_new + 'train_data_6.npy','wb')
np.save(f1,train_data)
f1.close()
f2 = file(path_new + 'test_data_6.npy','wb')
np.save(f2,test_data)
f2.close()
f3 = file(path_new + 'train_label_6.npy','wb')
np.save(f3,train_label)
f3.close()
f4 = file(path_new + 'test_label_6.npy','wb')
np.save(f4,test_label)
f4.close()

In [2]:
path_new = '/home/chocolate/LSTM-source/train_test_data/'
# 读取训练集和测试集
f1 = file(path_new + 'train_data_6.npy','rb')
train_data = np.load(f1)
f2 = file(path_new + 'test_data_6.npy','rb')
test_data = np.load(f2)
f3 = file(path_new + 'train_label_6.npy','rb')
train_label = np.load(f3)
f4 = file(path_new + 'test_label_6.npy','rb')
test_label = np.load(f4)

In [5]:
train_label.shape

(102500, 3)

In [7]:
import random

learning_rate = 0.001 # 学习速率
training_iters = 500000  # 训练总步长
batch_size = 25000 # 每小段步长
display_step = 1  # 每隔100小段显示输出

# 参数设置
n_input = 10 # 特征数量
n_steps = 15 # 时间序列长度
n_hidden = 500 # 隐藏层神经元个数
n_classes = 3 # 分类数量

tf.reset_default_graph()  # 重置流图
# 设置输入输出格式大小
xtr = tf.placeholder("float", [None, n_steps, n_input])
ytr = tf.placeholder("float", [None,n_classes])

# 定义权重和偏移量
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# def pred():
    
def RNN(x, weights, biases):

    # 更改数据集的格式，使其满足RNN网络的输入需求
    # 现在输入格式: (batch_size, n_steps, n_input)
    # 需要的格式: 'n_steps' 个tensor，每个的格式为 (batch_size, n_input)

    # 变换batch_size 和 n_steps的位置
    x = tf.transpose(x, [1, 0, 2])
    # 重新调整格式 (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # 分割成一系列的 'n_steps' 个tensors，每个的格式为 (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # 定义LSTM神经网络结构
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # 获取输出和状态
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)
    # 返回输出
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
# 获取预测值
pred = RNN(xtr, weights, biases)
# train_label = tf.split(0, batch_size, train_label)
# 取下一小段的输入
def next_batch_tr(batch_size):
    step = random.randint(1, len(train_data)/batch_size)
    next_batch_data = train_data[(step-1)*batch_size:step*batch_size]
    next_batch_label = train_label[(step-1)*batch_size:step*batch_size]   
    return next_batch_data, next_batch_label



In [ ]:
# 定义代价函数和最优化方法
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, ytr))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 评估模型准确率
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(ytr, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 变量初始化
init = tf.initialize_all_variables()
part_saver = tf.train.Saver()
# 创建会话
sess = tf.InteractiveSession()
sess.run(init)
step = 1
    # 一直迭代，直到最大步长
while step * batch_size < training_iters:
    batch_x, batch_y = next_batch_tr(batch_size)
    batch_x = batch_x.reshape((-1, n_steps, n_input))
        # 利用最优化方法调整权值
    sess.run(optimizer, feed_dict={xtr: batch_x, ytr: batch_y})
    if step % display_step == 0:
            # 计算准确率和损失函数
        acc = sess.run(accuracy, feed_dict={xtr: batch_x, ytr: batch_y})
        loss = sess.run(cost, feed_dict={xtr: batch_x, ytr: batch_y})
        # 每隔一定步长显示训练准确率和损失率
        print("Iter " + str(step * batch_size) + ", Minibatch Loss= " +
                "{:.6f}".format(loss) + ", Training Accuracy= " +
                "{:.5f}".format(acc))
    step += 1
print("Optimization Finished!")

saver_path = part_saver.save(sess, "/home/chocolate/LSTM-source/models/model_6/predict.ckpt")
print "Model saved." 


## 修改模型结构，如添加隐藏层数、添加dropout等

In [8]:
learning_rate = 0.001 # 学习速率
training_iters = 500000  # 训练总步长
batch_size = 50000 # 每小段步长
display_step = 1  # 每隔100小段显示输出

# 参数设置
n_input = 10 # 特征数量
n_steps = 15 # 时间序列长度
n_hidden = 500 # 隐藏层神经元个数
n_classes = 3 # 分类数量
num_layers = 2 #隐藏层数量
keep_prob = 1.0

# # Parameters
# learning_rate = 0.001
# training_iters = 30000
# batch_size = 128
# display_step = 10

# # Network Parameters
# n_input = 28 # MNIST data input (img shape: 28*28)
# n_steps = 28 # timesteps
# n_hidden = 128 # hidden layer num of features
# n_classes = 10 # MNIST total classes (0-9 digits)
# keep_prob = 1.0
# num_layers = 3 #隐藏层数量

tf.reset_default_graph()  # 重置流图
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'in':tf.Variable(tf.random_normal([n_input, n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'in':tf.Variable(tf.constant(0.1,shape = [n_hidden,])),
    'out': tf.Variable(tf.constant(0.1,shape = [n_classes,]))
}

In [9]:
def RNN(x, weights, biases):

# 输入层    
# x(batch_size,n_steps,n_input)
# ==>(batch_size * n_steps,n_input)
    x = tf.reshape(x, [-1, n_input])
# x_in==>(batch_size * n_steps,n_hidden)
    x_in = tf.matmul(x,weights['in']) + biases['in']
# x_in==>(batch_size,n_steps,n_hidden)
    x_in = tf.reshape(x_in,[-1,n_steps,n_hidden])

# 隐藏层
    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    _init_state = lstm_cell.zero_state(batch_size,dtype = tf.float32)
    # Get lstm cell output
    if keep_prob < 1: # 在外面包裹一层dropout
        lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * num_layers)
    outputs, states = tf.nn.dynamic_rnn(cell, x_in, dtype=tf.float32, time_major = False)

#输出层
    outputs = tf.unpack(tf.transpose(outputs,[1,0,2]))
    results = tf.matmul(outputs[-1],weights['out'])+biases['out']

    return results

pred = RNN(x, weights, biases)

# 取下一小段的输入
def next_batch_tr(batch_size):
    step = random.randint(1, len(train_data)/batch_size)
    next_batch_data = train_data[(step-1)*batch_size:step*batch_size]
    next_batch_label = train_label[(step-1)*batch_size:step*batch_size]   
    return next_batch_data, next_batch_label

# 定义代价函数和最优化方法
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 评估模型准确率
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 变量初始化
init = tf.initialize_all_variables()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        #  batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x, batch_y = next_batch_tr(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    
saver_path = part_saver.save(sess, "/home/chocolate/Model_LSTM-Future_20161123/models/model_6/predict.ckpt")
print "Model saved." 


ResourceExhaustedError: OOM when allocating tensor with shape[50000,500]
	 [[Node: RNN/while/MultiRNNCell/Cell0/BasicLSTMCell/split = Split[T=DT_FLOAT, num_split=2, _device="/job:localhost/replica:0/task:0/cpu:0"](RNN/while/MultiRNNCell/Cell0/BasicLSTMCell/split/split_dim, RNN/while/MultiRNNCell/Cell0/Slice)]]
Caused by op u'RNN/while/MultiRNNCell/Cell0/BasicLSTMCell/split', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python2.7/dist-pa

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import tensorflow as tf

from tensorflow.models.rnn.ptb import reader

flags = tf.flags
logging = tf.logging

flags.DEFINE_string(
    "model", "small",
    "A type of model. Possible options are: small, medium, large.")
flags.DEFINE_string("data_path", '/home/chocolate/download/simple-examples/data/', "data_path")
flags.DEFINE_bool("use_fp16", False,
                  "Train using 16-bit floats instead of 32bit floats")

FLAGS = flags.FLAGS


def data_type():
    return tf.float16 if FLAGS.use_fp16 else tf.float32


class PTBModel(object):
    """The PTB model."""

    def __init__(self, is_training, config):
        """
        :param is_training: 是否要进行训练.如果is_training=False,则不会进行参数的修正。
        """
        self.batch_size = batch_size = config.batch_size
        self.num_steps = num_steps = config.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size

        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])    # 输入
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps])       # 预期输出，两者都是index序列，长度为num_step
#         self._input_data = tf.placeholder("float", [None, n_steps, n_input])
#         self._targets = tf.placeholder("float", [None,n_classes])

        # Slightly better results can be obtained with forget gate biases
        # initialized to 1 but the hyperparameters of the model would need to be
        # different than reported in the paper.
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0, state_is_tuple=True)
        if is_training and config.keep_prob < 1: # 在外面包裹一层dropout
            lstm_cell = tf.nn.rnn_cell.DropoutWrapper(
                lstm_cell, output_keep_prob=config.keep_prob)
        cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers, state_is_tuple=True) # 多层lstm cell 堆叠起来

        self._initial_state = cell.zero_state(batch_size, data_type()) # 参数初始化,rnn_cell.RNNCell.zero_state

        with tf.device("/cpu:0"):
            embedding = tf.get_variable(
                "embedding", [vocab_size, size], dtype=data_type()) # vocab size * hidden size, 将单词转成embedding描述
            # 将输入seq用embedding表示, shape=[batch, steps, hidden_size]
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)

        # Simplified version of tensorflow.models.rnn.rnn.py's rnn().
        # This builds an unrolled LSTM for tutorial purposes only.
        # In general, use the rnn() or state_saving_rnn() from rnn.py.
        #
        # The alternative version of the code below is:
        #
        # inputs = [tf.squeeze(input_, [1])
        #           for input_ in tf.split(1, num_steps, inputs)]
        # outputs, state = tf.nn.rnn(cell, inputs, initial_state=self._initial_state)
        outputs = []
        state = self._initial_state # state 表示 各个batch中的状态
        with tf.variable_scope("RNN"):
            for time_step in range(num_steps):
                if time_step > 0: tf.get_variable_scope().reuse_variables()
                # cell_out: [batch, hidden_size]
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                outputs.append(cell_output)  # output: shape[num_steps][batch,hidden_size]

        # 把之前的list展开，成[batch, hidden_size*num_steps],然后 reshape, 成[batch*numsteps, hidden_size]
        output = tf.reshape(tf.concat(1, outputs), [-1, size])

        # softmax_w , shape=[hidden_size, vocab_size], 用于将distributed表示的单词转化为one-hot表示
        softmax_w = tf.get_variable(
            "softmax_w", [size, vocab_size], dtype=data_type())
        softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=data_type())
        # [batch*numsteps, vocab_size] 从隐藏语义转化成完全表示
        logits = tf.matmul(output, softmax_w) + softmax_b

        # loss , shape=[batch*num_steps]
        # 带权重的交叉熵计算
        loss = tf.nn.seq2seq.sequence_loss_by_example(
            [logits],   # output [batch*numsteps, vocab_size]
            [tf.reshape(self._targets, [-1])],  # target, [batch_size, num_steps] 然后展开成一维【列表】
            [tf.ones([batch_size * num_steps], dtype=data_type())]) # weight
        self._cost = cost = tf.reduce_sum(loss) / batch_size # 计算得到平均每批batch的误差
        self._final_state = state

        if not is_training:  # 如果没有训练，则不需要更新state的值。
            return

        self._lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        # clip_by_global_norm: 梯度衰减，具体算法为t_list[i] * clip_norm / max(global_norm, clip_norm)
        # 这里gradients求导，ys和xs都是张量
        # 返回一个长为len(xs)的张量，其中的每个元素都是\grad{\frac{dy}{dx}}
        # clip_by_global_norm 用于控制梯度膨胀,前两个参数t_list, global_norm, 则
        # t_list[i] * clip_norm / max(global_norm, clip_norm)
        # 其中 global_norm = sqrt(sum([l2norm(t)**2 for t in t_list]))
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                          config.max_grad_norm)

        # 梯度下降优化，指定学习速率
        optimizer = tf.train.GradientDescentOptimizer(self._lr)
        # optimizer = tf.train.AdamOptimizer()
        # optimizer = tf.train.GradientDescentOptimizer(0.5)
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))  # 将梯度应用于变量

        self._new_lr = tf.placeholder(
            tf.float32, shape=[], name="new_learning_rate")     #   用于外部向graph输入新的 lr值
        self._lr_update = tf.assign(self._lr, self._new_lr)     #   使用new_lr来更新lr的值

    def assign_lr(self, session, lr_value):
        # 使用 session 来调用 lr_update 操作
        session.run(self._lr_update, feed_dict={self._new_lr: lr_value})

    @property
    def input_data(self):
        return self._input_data

    @property
    def targets(self):
        return self._targets

    @property
    def initial_state(self):
        return self._initial_state

    @property
    def cost(self):
        return self._cost

    @property
    def final_state(self):
        return self._final_state

    @property
    def lr(self):
        return self._lr

    @property
    def train_op(self):
        return self._train_op


class SmallConfig(object):
    """Small config."""
    init_scale = 0.1        #
    learning_rate = 1.0     # 学习速率
    max_grad_norm = 5       # 用于控制梯度膨胀，
    num_layers = 2          # lstm层数
    num_steps = 20          # 单个数据中，序列的长度。
    hidden_size = 200       # 隐藏层规模
    max_epoch = 4           # epoch<max_epoch时，lr_decay值=1,epoch>max_epoch时,lr_decay逐渐减小
    max_max_epoch = 13      # 指的是整个文本循环13遍。
    keep_prob = 1.0
    lr_decay = 0.5          # 学习速率衰减
    batch_size = 20         # 每批数据的规模，每批有20个。
    vocab_size = 10000      # 词典规模，总共10K个词


class MediumConfig(object):
    """Medium config."""
    init_scale = 0.05
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 35
    hidden_size = 650
    max_epoch = 6
    max_max_epoch = 39
    keep_prob = 0.5
    lr_decay = 0.8
    batch_size = 20
    vocab_size = 10000


class LargeConfig(object):
    """Large config."""
    init_scale = 0.04
    learning_rate = 1.0
    max_grad_norm = 10
    num_layers = 2
    num_steps = 35
    hidden_size = 1500
    max_epoch = 14
    max_max_epoch = 55
    keep_prob = 0.35
    lr_decay = 1 / 1.15
    batch_size = 20
    vocab_size = 10000


class TestConfig(object):
    """Tiny config, for testing."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 1
    num_layers = 1
    num_steps = 2
    hidden_size = 2
    max_epoch = 1
    max_max_epoch = 1
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000


def run_epoch(session, model, data, eval_op, verbose=False):
    """Runs the model on the given data."""
    # epoch_size 表示批次总数。也就是说，需要向session喂这么多次数据
    epoch_size = ((len(data) // model.batch_size) - 1) // model.num_steps  # // 表示整数除法
    start_time = time.time()
    costs = 0.0
    iters = 0
    state = session.run(model.initial_state)
    for step, (x, y) in enumerate(reader.ptb_iterator(data, model.batch_size,
                                                      model.num_steps)):
        fetches = [model.cost, model.final_state, eval_op] # 要进行的操作，注意训练时和其他时候eval_op的区别
        feed_dict = {}      # 设定input和target的值
        feed_dict[model.input_data] = x
        feed_dict[model.targets] = y
        for i, (c, h) in enumerate(model.initial_state):
            feed_dict[c] = state[i].c   # 这部分有什么用？看不懂
            feed_dict[h] = state[i].h
        cost, state, _ = session.run(fetches, feed_dict) # 运行session,获得cost和state
        costs += cost   # 将 cost 累积
        iters += model.num_steps

        if verbose and step % (epoch_size // 10) == 10:  # 也就是每个epoch要输出10个perplexity值
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                  (step * 1.0 / epoch_size, np.exp(costs / iters),
                   iters * model.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)


def get_config():
    if FLAGS.model == "small":
        return SmallConfig()
    elif FLAGS.model == "medium":
        return MediumConfig()
    elif FLAGS.model == "large":
        return LargeConfig()
    elif FLAGS.model == "test":
        return TestConfig()
    else:
        raise ValueError("Invalid model: %s", FLAGS.model)


# def main(_):
if __name__=='__main__':
    if not FLAGS.data_path:
        raise ValueError("Must set --data_path to PTB data directory")
    print(FLAGS.data_path)

    raw_data = reader.ptb_raw_data(FLAGS.data_path) # 获取原始数据
    train_data, valid_data, test_data, _ = raw_data

    config = get_config()
    eval_config = get_config()
    eval_config.batch_size = 1
    eval_config.num_steps = 1

    with tf.Graph().as_default(), tf.Session() as session:
        initializer = tf.random_uniform_initializer(-config.init_scale, # 定义如何对参数变量初始化
                                                    config.init_scale)
        with tf.variable_scope("model", reuse=None,initializer=initializer):
            m = PTBModel(is_training=True, config=config)   # 训练模型， is_trainable=True
        with tf.variable_scope("model", reuse=True,initializer=initializer):
            mvalid = PTBModel(is_training=False, config=config) #  交叉检验和测试模型，is_trainable=False
            mtest = PTBModel(is_training=False, config=eval_config)

        summary_writer = tf.train.SummaryWriter('/tmp/lstm_logs',session.graph)

        tf.initialize_all_variables().run()  # 对参数变量初始化

        for i in range(config.max_max_epoch):   # 所有文本要重复多次进入模型训练
            # learning rate 衰减
            # 在 遍数小于max epoch时， lr_decay = 1 ; > max_epoch时， lr_decay = 0.5^(i-max_epoch)
            lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
            m.assign_lr(session, config.learning_rate * lr_decay) # 设置learning rate

            print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
            train_perplexity = run_epoch(session, m, train_data, m.train_op,verbose=True) # 训练困惑度
            print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
            valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op()) # 检验困惑度
            print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

        test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())  # 测试困惑度
        print("Test Perplexity: %.3f" % test_perplexity)


# if __name__ == "__main__":
#     tf.app.run()